First, import needed modules

In [2]:
import numpy as np
import pandas as pd
import os

Initalize variables

In [ ]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [16]:
btw13_es = pd.read_csv(f'{path}/data/elections/BTW13_Erststimmen_Wahlbezirke.csv', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw13_es.head()

/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_23579/4294177193.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  btw13_es = pd.read_csv(f'{path}/data/elections/BTW13_Erststimmen_Wahlbezirke.csv', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})


,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk,Bezirksart,Wahlberechtigte (A),...,BIG,pro Deutschland,FREIE WÄHLER,PARTEI DER VERNUNFT,Die PARTEI,Bergpartei,BGD,DKP,NEIN,WGr/EB
0,1,01,0,01,0,000,0,001,0,1701,...,0,0,0,0,0,0,0,0,0,0
1,1,01,0,01,0,000,0,002,0,1567,...,0,0,0,0,0,0,0,0,0,0
2,1,01,0,01,0,000,0,003,0,1670,...,0,0,0,0,0,0,0,0,0,0
3,1,01,0,01,0,000,0,004,0,1592,...,0,0,0,0,0,0,0,0,0,0
4,1,01,0,01,0,000,0,005,0,1277,...,0,0,0,0,0,0,0,0,0,0


Clean dataset

In [19]:
btw13_es = btw13_es[btw13_es['Bezirksart'] == 0] # keeps only Urnenwahlbezirke
btw13_es.head()

,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk,Bezirksart,Wahlberechtigte (A),...,pro Deutschland,FREIE WÄHLER,PARTEI DER VERNUNFT,Die PARTEI,Bergpartei,BGD,DKP,NEIN,WGr/EB,AGS
0,1,01,0,01,0,000,0,001,0,1701,...,0,0,0,0,0,0,0,0,0,01001000
1,1,01,0,01,0,000,0,002,0,1567,...,0,0,0,0,0,0,0,0,0,01001000
2,1,01,0,01,0,000,0,003,0,1670,...,0,0,0,0,0,0,0,0,0,01001000
3,1,01,0,01,0,000,0,004,0,1592,...,0,0,0,0,0,0,0,0,0,01001000
4,1,01,0,01,0,000,0,005,0,1277,...,0,0,0,0,0,0,0,0,0,01001000


Create the AGS, it follows the scheme ll r kk ggg (Land, Regierunsbezirk, Kreis, Gemeinde) -> combine these columns

In [23]:
btw13_es['AGS'] = btw13_es['Land'] + btw13_es['Regierungsbezirk'] + btw13_es['Kreis'] + btw13_es['Gemeinde']
btw13_es.head()
btw13_es.dtypes['AGS']


dtype('O')

Merge with treatment dataset

In [25]:
treatment = pd.read_csv(f'{path}/data/treatment.csv')
treatment = treatment.astype({"AGS": object})
data = pd.merge(btw13_es, treatment, on='AGS', how='output', indicator=True)
data.head()

KeyError: 'output'